In [1]:
from pyspark.sql import SparkSession

In [2]:
# Out of Memory 시 지정
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("movie-recommendation")\
    .config("spark.executor.memory", MAX_MEMORY)\
    .config("spark.driver.memory", MAX_MEMORY)\
    .getOrCreate()

21/12/16 01:50:41 WARN Utils: Your hostname, 6miniui-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.200.112 instead (on interface en0)
21/12/16 01:50:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/16 01:50:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 레이팅스 데이터 프레임 생성
ratings_df = spark.read.csv("/Users/6mini/ml-25m/ratings.csv", inferSchema=True, header=True)

In [4]:
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [5]:
# 타임 스탬프 컬럼은 지운다.
ratings_df = ratings_df.select(["userId", "movieId", "rating"])

In [6]:
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [7]:
# 레이팅의 통계를 내본다.
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423535|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [8]:
# 트레인 테스트셋 분리
train_df, test_df = ratings_df.randomSplit([0.8, 0.2])

In [9]:
from pyspark.ml.recommendation import ALS

In [10]:
# ALS 이용 모델링
als = ALS(
    maxIter=5, # 반복 횟수
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop" # 학습되지 못한 데이터의 처리 방법
)

In [11]:
# 학습
model = als.fit(train_df)

21/12/16 01:55:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/12/16 01:55:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
21/12/16 01:55:49 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [12]:
# 예측
predictions = model.transform(test_df)

In [13]:
predictions.show()
# 생각보다 잘 맞춘 것 같다.

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   321|   3175|   3.0| 3.2872846|
|   321|   6620|   3.5| 3.6733642|
|   322|    463|   3.0|  3.269444|
|   368|   1580|   3.5| 3.6708436|
|   368|  54190|   4.0| 3.4968219|
|   375|   1580|   2.5|  3.421627|
|   458|   1580|   3.5| 3.1025493|
|   497|   2366|   4.0| 3.8671935|
|   588|   1580|   2.5| 2.6639755|
|   596|   1580|   3.0| 3.5359857|
|   597|   1088|   3.0| 3.2686214|
|   597|   1580|   4.0| 3.6997573|
|   597|   3997|   1.0| 1.9885403|
|   613|   1645|   4.0| 3.7589316|
|   756|  44022|   3.5|  3.107243|
|   847|   4818|   0.5| 1.4983841|
|   847|   7833|   3.0| 3.6151803|
|   847|  96488|   4.0| 3.8599513|
|   847| 180981|   3.5| 3.4091368|
|   879|  68135|   5.0| 4.4257684|
+------+-------+------+----------+
only showing top 20 rows



In [14]:
# 예측값과 실제값의 통계를 확인한다.
predictions.select('rating', 'prediction').describe().show()
# 

+-------+-----------------+------------------+
|summary|           rating|        prediction|
+-------+-----------------+------------------+
|  count|          4997993|           4997993|
|   mean|3.533390903108508| 3.423480612159125|
| stddev|1.060723007511501|0.6444551403360655|
|    min|              0.5|        -1.6993694|
|    max|              5.0|         6.4476376|
+-------+-----------------+------------------+



In [15]:
# rmse로 이밸류에이션을 진행한다.
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol='rating', predictionCol='prediction')

In [16]:
rmse = evaluator.evaluate(predictions)

In [17]:
print(rmse)

0.8087562307490825


In [18]:
# 레코멘데이션을 직접 뽑아본다.
# 유저마다 탑 3개의 아이템을 추천한다.
model.recommendForAllUsers(3).show()

/Users/6mini/opt/anaconda3/envs/spark-flink/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[{194434, 7.50267...|
|    31|[{203882, 3.97501...|
|    34|[{194434, 5.53526...|
|    53|[{192089, 6.53624...|
|    65|[{194434, 6.70630...|
|    78|[{203882, 6.70107...|
|    81|[{192689, 5.35233...|
|    85|[{203086, 5.54420...|
|   101|[{203882, 5.29644...|
|   108|[{194434, 5.46318...|
|   115|[{203882, 6.33117...|
|   126|[{203086, 6.45976...|
|   133|[{203086, 5.32927...|
|   137|[{203086, 5.65758...|
|   148|[{194434, 5.89812...|
|   155|[{194434, 5.75075...|
|   183|[{194434, 5.71291...|
|   193|[{183947, 5.34912...|
|   210|[{126941, 7.52532...|
|   211|[{203086, 6.51664...|
+------+--------------------+
only showing top 20 rows



In [19]:
# 아이템마다 탑 3개의 유저를 추천한다.
model.recommendForAllItems(3).show()
# 영화마다 유저를 추천해준다.

/Users/6mini/opt/anaconda3/envs/spark-flink/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     28|[{105801, 5.53363...|
|     31|[{87426, 5.40749}...|
|     34|[{128562, 5.47395...|
|     53|[{7552, 5.1105866...|
|     65|[{87426, 5.288509...|
|     78|[{142811, 4.78454...|
|     81|[{105946, 4.77834...|
|     85|[{160416, 4.85580...|
|    101|[{57450, 4.929262...|
|    108|[{34485, 6.031539...|
|    115|[{142811, 5.85976...|
|    126|[{87426, 4.779694...|
|    133|[{142811, 5.69091...|
|    137|[{142811, 5.29639...|
|    148|[{160416, 4.21056...|
|    155|[{10417, 5.080611...|
|    183|[{10417, 5.191336...|
|    193|[{87426, 5.088001...|
|    210|[{87426, 4.919769...|
|    211|[{105801, 5.12127...|
+-------+--------------------+
only showing top 20 rows



In [20]:
# 실제 서비스에 이렇게 쓰이지 않는다 API로 특정 유저를 위한 추천을 불러오게 만들 것이다.
from pyspark.sql.types import IntegerType

user_list = [65, 78, 81]
# 데이터 프레임 생성
users_df = spark.createDataFrame(user_list, IntegerType()).toDF('userId')

users_df.show()

+------+
|userId|
+------+
|    65|
|    78|
|    81|
+------+



In [ ]:
# 위 데이터 프레임을 모델 추천에 넣어주면 된다.
user_recs = model.recommendForUserSubset(users_df, 5)
# 유저마다 레코멘테이션이 5개씩 전시된다.
user_recs.collect()

In [22]:
# 이름으로 바꾸기 위해 리스트를 만든다.
# 첫번째 값만 꺼내온다.
movies_list = user_recs.collect()[0].recommendations

In [23]:
# 리스트 데이터를 스파크 데이터 프레임으로 변경한다.
recs_df = spark.createDataFrame(movies_list)
recs_df.show()


+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 194434|  6.70630407333374|
| 126941| 6.530391216278076|
| 203882|   6.5257887840271|
| 203086| 6.447256088256836|
| 205453|6.0838799476623535|
+-------+------------------+



In [24]:
# 무비 id를 이름으로 변경하기 위한 데이터를 로드한다.
movies_df = spark.read.csv("/Users/6mini/ml-25m/movies.csv", inferSchema=True, header=True)

In [25]:
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [26]:
# SQL로 조인하기 위해 TempView에 담는다.
recs_df.createOrReplaceTempView("recommendations")
movies_df.createOrReplaceTempView("movies")

In [27]:
query = """
SELECT *
FROM
    movies JOIN recommendations
    ON movies.movieId = recommendations.movieId
ORDER BY
    rating desc
"""
recommended_movies = spark.sql(query)
recommended_movies.show()

+-------+--------------------+------------------+-------+------------------+
|movieId|               title|            genres|movieId|            rating|
+-------+--------------------+------------------+-------+------------------+
| 194434|   Adrenaline (1990)|(no genres listed)| 194434|  6.70630407333374|
| 126941|Joni's Promise (2...|            Comedy| 126941| 6.530391216278076|
| 203882|Dead in the Water...|            Horror| 203882|   6.5257887840271|
| 203086|Truth and Justice...|             Drama| 203086| 6.447256088256836|
| 205453|The Good Fight: T...|       Documentary| 205453|6.0838799476623535|
+-------+--------------------+------------------+-------+------------------+



In [28]:
# 실제 서비스할 때 하나의 함수로 사용하면 좋으니, 서빙하는 함수를 생성해본다.
def get_recommendations(user_id, num_recs): # 유저 아이디와 추천 수
    users_df = spark.createDataFrame([user_id], IntegerType()).toDF('userId')
    user_recs_df = model.recommendForUserSubset(users_df, num_recs)
    
    recs_list = user_recs_df.collect()[0].recommendations
    recs_df = spark.createDataFrame(recs_list)
    recommended_movies = spark.sql(query)
    return recommended_movies

In [29]:
recs = get_recommendations(456, 10)

/Users/6mini/opt/anaconda3/envs/spark-flink/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [30]:
recs.toPandas()

,movieId,title,genres,movieId,rating
0,194434,Adrenaline (1990),(no genres listed),194434,6.706304
1,126941,Joni's Promise (2005),Comedy,126941,6.530391
2,203882,Dead in the Water (2006),Horror,203882,6.525789
3,203086,Truth and Justice (2019),Drama,203086,6.447256
4,205453,The Good Fight: The Abraham Lincoln Brigade in...,Documentary,205453,6.083880


In [31]:
spark.stop()